In [1]:
import spacy
from spacy.matcher import Matcher #class pour réalisation du matching
from spacy.matcher import PhraseMatcher
from spacy.language import Language
from spacy.tokens import Span #class pour extraction des mots
from joblib import Parallel, delayed
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.symbols import nsubj, VERB
from spacy import displacy

In [5]:
nlp_fr = spacy.load('fr_core_news_sm') 

print (spacy.__version__)
print("\n")

# customization spacy 
#nlp_fr.add_pipe("lemmatizer", config = {"mode": "lookup"})
nlp_fr.add_pipe("sentencizer")

nlp_fr.pipeline
print(nlp_fr.pipeline, sep='\n')
print("\n")
analysis = nlp_fr.analyze_pipes(pretty = True)

if spacy.prefer_gpu():
    print("Working on GPU.")
else:
    print("No GPU found, working on CPU.")

3.1.4


[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7fdd176f6ea0>), ('morphologizer', <spacy.pipeline.morphologizer.Morphologizer object at 0x7fdd176d84a0>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7fdd1762fee0>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7fdd1770b380>), ('lemmatizer', <spacy.lang.fr.lemmatizer.FrenchLemmatizer object at 0x7fdd15851180>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7fdd176e0ac0>), ('sentencizer', <spacy.pipeline.sentencizer.Sentencizer object at 0x7fdd25dc1640>)]



============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                     

In [2]:
import pandas as pd

In [3]:
raw_data = pd.read_excel(r'./raw_data.xlsx', 'Sheet1')

In [4]:
# transformation de la structure du dataframe 
## pivot des commentaires 
mod_data = (raw_data.set_index(['titre', 'corps', 'nombre_de_commentaires', 'nombre_de_soutiens', "nombre_abonnements", 'categorie'])
   .rename_axis(['drop'], axis=1)
   .stack()
   .reset_index())
mod_data = mod_data.drop(['drop'], axis=1)
mod_data = mod_data.rename(columns = {mod_data.columns[6]: "commentaires" })

## suppression des commentaires ayant comme texte "Pas de commentaies"
mod_data = mod_data[~mod_data['commentaires'].str.endswith('commentaires')]

In [6]:
%%time
nlp_fr.max_length = 6100000
text =  mod_data['commentaires'].values
text = ' '.join(map(str, text))
docs = nlp_fr(text)


CPU times: user 32.2 s, sys: 2.28 s, total: 34.5 s
Wall time: 35.2 s


In [7]:
text = [token for token in docs]

In [8]:
set([w.label_ for w in docs.ents])

{'LOC', 'MISC', 'ORG', 'PER'}

In [10]:
from collections import Counter
entites = [ent.text.lower() for ent in docs.ents if ent.text.lower() not in STOP_WORDS]
entites_count = Counter(entites).most_common(50)
entites_count

[('etat', 60),
 ('france', 56),
 ('la france', 45),
 ('cour', 42),
 ('bloom', 41),
 ('cour des comptes', 38),
 ('smd3', 21),
 ('bonjour', 20),
 ('grand paris', 17),
 ('caf', 16),
 ('cdc', 15),
 ('état', 13),
 ('europe', 13),
 ('français', 12),
 ('sgp', 12),
 ('paris', 12),
 ('ime', 12),
 ('allemagne', 11),
 ('ofb', 10),
 ('edf', 10),
 ('gonesse', 10),
 ('meudon', 10),
 ('ars', 10),
 ('macron', 9),
 ('saclay', 9),
 ('ugap', 9),
 ('ifrap', 9),
 ('sénat', 8),
 ('rte', 8),
 ('covid', 8),
 ('libreoffice', 8),
 ('pav', 8),
 ('at', 8),
 ('amp', 7),
 ('ca', 7),
 ('gpe', 7),
 ('gafam', 7),
 ('pnr', 7),
 ('cir', 7),
 ('cnav', 7),
 ("l'europe", 6),
 ('de france', 6),
 ('grand paris express', 6),
 ('tva', 6),
 ('dgfip', 6),
 ('assemblée nationale', 6),
 ('merci bloom', 5),
 ('acca', 5),
 ('rer d', 5),
 ('rer b', 5)]

In [11]:
entites = [ent.text.lower() for ent in docs.ents if ent.text.lower() not in STOP_WORDS and ent.label_ == "PER"]
entites_count = Counter(entites).most_common(50)
entites_count

[('bloom', 24),
 ('bonjour', 8),
 ('macron', 6),
 ('mc kinsey', 4),
 ('willy schraen', 4),
 ('bravo', 4),
 ('soutien', 3),
 ('merci bloom', 3),
 ('honte', 3),
 ('jean vivier', 3),
 ('fonctionnaires', 3),
 ('md€', 3),
 ('claire', 2),
 ('l’état', 2),
 ('florian', 2),
 ('ben', 2),
 ('andré mugnier', 2),
 ('le cérémé', 2),
 ('rte', 2),
 ('alain rouquier-perret', 2),
 ('premier ministre', 2),
 ('jacqueline lorthiois', 2),
 ('jacqueline', 2),
 ('dominique dufumier', 2),
 ('diminuer', 2),
 ('jean-pierre orfeuil', 2),
 ('charles prats', 2),
 ('dépenses', 2),
 ('brillants', 2),
 ('pfizer', 2),
 ('pascal richet', 2),
 ('rse', 2),
 ('h.\n', 2),
 ('valentin haüy', 2),
 ('idem', 2),
 ('balladur', 2),
 ('chloé morin', 2),
 ('hyper', 2),
 ('proteger', 1),
 ('protéger', 1),
 ('réellement', 1),
 ('notre président', 1),
 ('première ministre', 1),
 ('yvette ramos', 1),
 ('claire nouvian', 1),
 ('demain', 1),
 ('jan onyme', 1),
 ('l’océan', 1),
 ('l’onu', 1),
 ('l’aquaculture', 1)]

In [ ]:
entites = [ent.text.lower() for ent in docs.ents if ent.text.lower() not in STOP_WORDS and ent.label_ == "MISC"]
entites_count = Counter(entites).most_common(50)
entites_count

In [13]:
# instantiation du matcher 
matcher = Matcher(nlp_fr.vocab) 

# definition du pattern de recherche
pattern = [
    [{"LOWER": "cour"}, {"LOWER": "des"},{"LOWER": "comptes"}],
 [{"LOWER": "cour"}, {"LOWER": "des"},{"LOWER": "Comptes"}]
]


base_data = mod_data['commentaires'].str.strip().values
base_data  = ' '.join(map(str, base_data ))

# ajout du pattern au matcher
matcher.add("Matching", pattern)
mod_doc = nlp_fr(base_data)
matches = matcher(mod_doc) 

for match_id, start, end in matches:
    # match_id
    string_id = nlp_fr.vocab.strings[match_id]  
    # position début/fin du résultat 
    span = docs[start:end] 
    print(match_id, string_id, start, end, span.text)
    #print(span.text)

6895354335150655416 Matching 276 279 Cour des Comptes
6895354335150655416 Matching 299 302 Cour des Comptes
6895354335150655416 Matching 822 825 Cour des comptes
6895354335150655416 Matching 2059 2062 cour des comptes
6895354335150655416 Matching 2658 2661 Cour des Comptes
6895354335150655416 Matching 2915 2918 Cour des Comptes
6895354335150655416 Matching 2935 2938 Cour des Comptes
6895354335150655416 Matching 3887 3890 Cour des Comptes
6895354335150655416 Matching 4702 4705 Cour des Comptes
6895354335150655416 Matching 4988 4991 Cour des Comptes
6895354335150655416 Matching 5019 5022 cour des comptes
6895354335150655416 Matching 7083 7086 Cour des Comptes
6895354335150655416 Matching 7307 7310 cour des comptes
6895354335150655416 Matching 7478 7481 Cour des Comptes
6895354335150655416 Matching 7547 7550 cour des comptes
6895354335150655416 Matching 7663 7666 Cour des Comptes
6895354335150655416 Matching 8332 8335 Cour des comptes
6895354335150655416 Matching 12933 12936 cour des comp

In [14]:
spacy_adj = [tok for tok in docs if tok.pos_ == 'ADJ' and not tok.is_stop and not tok.is_punct] #adjectifs
spacy_adj_count = Counter([tok.text.lower() for tok in spacy_adj]).most_common(20)
spacy_adj_count

[('public', 159),
 ('publics', 96),
 ('publique', 77),
 ('publiques', 70),
 ('grand', 60),
 ('nécessaire', 47),
 ('français', 45),
 ('bon', 40),
 ('grande', 40),
 ('bonne', 39),
 ('climatique', 37),
 ('écologique', 34),
 ('sociale', 32),
 ('agricoles', 32),
 ('nationale', 29),
 ('particulier', 28),
 ('économique', 28),
 ('cher', 27),
 ('efficace', 26),
 ('privés', 26)]

In [15]:
spacy_verb = [tok for tok in docs if tok.pos_ == 'VERB' and not tok.is_punct and not tok.is_stop] #verbes # and not tok.is_stop
spacy_verb_count = Counter([tok.text.strip() for tok in spacy_verb]).most_common(30)
spacy_verb_count

[('faire', 157),
 ('faut', 134),
 ('devrait', 73),
 ('mettre', 59),
 ('pouvoir', 40),
 ('voir', 37),
 ('faudrait', 36),
 ('devraient', 35),
 ('agit', 33),
 ('demande', 31),
 ('existe', 30),
 ('savoir', 29),
 ('veut', 27),
 ('financer', 26),
 ('contrôler', 25),
 ('payer', 25),
 ('demander', 24),
 ('prendre', 24),
 ('passer', 22),
 ('donner', 21),
 ('permettre', 21),
 ('mis', 20),
 ('privée', 20),
 ('compter', 20),
 ('utilisé', 19),
 ('aller', 19),
 ('soutiens', 19),
 ('justifier', 19),
 ('gérer', 19),
 ('détruire', 19)]

In [16]:
spacy_noun = [tok for tok in docs if tok.pos_ == 'NOUN' and not tok.is_punct and not tok.is_stop] #verbes # and not tok.is_stop
spacy_noun_count = Counter([tok.text.strip() for tok in spacy_noun]).most_common(30)
spacy_noun_count

[('argent', 199),
 ('comptes', 117),
 ('système', 101),
 ('personnes', 93),
 ('Cour', 92),
 ('chasse', 92),
 ('ans', 84),
 ('travail', 79),
 ('vie', 76),
 ('temps', 76),
 ('chasseurs', 73),
 ('état', 72),
 ('compte', 72),
 ('biodiversité', 70),
 ('service', 70),
 ('proposition', 69),
 ('exemple', 69),
 ('Etat', 68),
 ('cas', 67),
 ('services', 66),
 ('projet', 62),
 ('citoyens', 61),
 ('coût', 61),
 ('intérêt', 59),
 ('ligne', 59),
 ('utilisation', 56),
 ('terres', 56),
 ('place', 56),
 ('rapport', 56),
 ('part', 55)]

In [19]:
cc_sentences = [sent for sent in docs.sents if 'Comptes' in sent.text]
cc_sentences 

[La protection des mers et des océans est une priorité absolue et la Cour des Comptes doit enquêter et informer les citoyens sur l'utilisation de ces subventions.,
 Une mission prioritaire pour la Cour des Comptes !,
 Les rapports de la Cour des Comptes sont très importants.,
 
 
 Et puisque la Cour des Comptes contrôle les comptes de l'état, pourquoi n'y aurait-il pas au sein de la Cour des Comptes une équipe chargée de contrôler l'éventuel greenwashing de l'état ?,
 Oui, que la Cour des Comptes aille regarder ça de près et fasse cesser la prédation de l'océan et des finances publiques.,
 La Cour des Comptes peut-elle mettre en cause la responsabilité du ministre et du directeur des pêches maritimes et de l'aquaculture pour que les documents réclamés par l'association BLOOM soient enfin communiqués et exploitables ?,
 Merci à Bloom d'insister en notre nom, et à la Cour des Comptes de bien vouloir exercer sa compétence.,
 je soutiens la démarche de l'association BLOOM qui est de demand

In [22]:
def adjectivesDescribingCharacters(text, character):
    sents = [sent for sent in text.sents if character in sent.text] # filtrage sur les phrases
    adjectives = []
    for sent in sents: 
        for word in sent: 
            if character in word.text: # filtrage sur les mots
                for child in word.children: # filtrage sur les mots 
                    if child.pos_ == 'ADJ' and not child.is_stop :  # filtrage sur les mots #and not child.is_stop
                        adjectives.append(child.text.strip())
    return Counter(adjectives).most_common(10)

In [24]:
comptes_verbs = adjectivesDescribingCharacters(docs, 'Comptes')
comptes_verbs 

[('aille', 1)]

In [ ]:
def verbsToMatrix(verbCounts): 
    return pd.Series({t[0]: t[1] for t in verbCounts})